In [1]:
from collections import defaultdict
import random
import subprocess
import json
import tqdm
import concurrent.futures
import plotly.graph_objects as go
import re
import os

In [ ]:
# aplica na população e marca os indivíduos
# aplica em uma porcentagem baixa dos demais indivíduos
# gerar vetor de pares nas buscas locais

In [30]:
params = {
    "p": 200, # [200, 300, 400, 500]
    "pe": 0.10, # [0.05, 0.10, 0.15]
    "pm": 0.10, # [0.10, 0.20, 0.30]
    "rhoe": 0.60, # [0.50, 0.55, 0.60, 0.65]
    "K": 1, # [1]
    "MAXT": 10, # [-]
    "X_INTVL": 1, # [-]
    "X_NUMBER": 0, # []
    "MAX_GENS": 1000, # [5000]
    "irace": "false",
    "rngSeed": 2147483647
}

In [3]:
class OutputReport:
    results: dict
    
    def __init__(self, results):
        self.results = results

    def plot_convergence(self):
        fig = go.Figure()

        for instance, instance_results in self.results.items():
            # Get the result with the longest CONVERGENCE_REPORT
            best_result = max(instance_results, key=lambda c: len(c["CONVERGENCE_REPORT"]))
            convergence_report = best_result["CONVERGENCE_REPORT"]
            
            # Extract generations and best_fitness values
            generations = [entry["generation"] for entry in convergence_report]
            best_fitness = [entry["best_fitness"] for entry in convergence_report]

            # Add a trace for each instance
            fig.add_trace(go.Scatter(
                x=generations, 
                y=best_fitness, 
                mode='lines+markers', 
                name=f'Instance {instance}',
                line=dict(width=2),
                marker=dict(size=6),
                hoverinfo="x+y+name"
            ))

        # Improve layout aesthetics
        fig.update_layout(
            title="Convergence Report for All Instances",
            xaxis_title="Generation",
            yaxis_title="Best Fitness",
            template="plotly_dark",
            hovermode="x unified",
            font=dict(size=14),
            margin=dict(l=50, r=50, t=50, b=50),
            width=900, height=550,
            legend=dict(title="Instances", bgcolor="rgba(0,0,0,0.5)")
        )

        fig.show()
    
    def plot_comparison(self):
        comparison = {
            instance: {
                "BRKGA": int(min(r["RESULT"]["Min_f(x)"] for r in instance_results)),
                "BRKGA Time": sum(r["EXECUTION_TIME_REPORT"]["duration_seconds"] for r in instance_results) / len(instance_results),
                "SOA": instance_results[0]["INSTANCE"]["SOA_Cost"]
            }
            for instance, instance_results in self.results.items()
        }
        # Function to extract the numerical part from the instance name
        def extract_number(instance_name):
            match = re.search(r'(\d+)', instance_name)  # Look for the first sequence of digits
            return int(match.group(1)) if match else float('inf')  # Return the number or inf if no match

        # Sort instance names based on the numerical part
        instance_names = sorted(comparison.keys(), key=extract_number)
        brkga_values = [comparison[instance]["BRKGA"] for instance in instance_names]
        soa_values = [comparison[instance]["SOA"] for instance in instance_names]
        
        brkga_colors = []
        soa_colors = []
        
        for brkga, soa in zip(brkga_values, soa_values):
            if brkga == soa:
                brkga_colors.append('yellow')
                soa_colors.append('yellow')
            else:
                if brkga > soa:
                    brkga_colors.append('red')
                    soa_colors.append('blue')
                else:
                    brkga_colors.append('blue')
                    soa_colors.append('red')
        
        # Create the grouped bar plot
        fig = go.Figure(data=[
            go.Bar(
                x=instance_names, 
                y=brkga_values,
                name="BRKGA",
                hovertext=[f"BRKGA: {comparison[instance]['BRKGA']}" for instance in instance_names],
                hoverinfo='text',
                marker=dict(color=brkga_colors)
            ),
            go.Bar(
                x=instance_names,
                y=soa_values,
                name="SOA",
                hovertext=[f"SOA: {soa}" for soa in soa_values],
                hoverinfo='text',
                marker=dict(color=soa_colors)
            )
        ])

        # Update layout for better visualization
        fig.update_layout(
            title='Comparison of BRKGA and SOA Costs for Each Instance',
            xaxis_title='Instance Name',
            yaxis_title='Cost',
            barmode='group',  # Group the bars side by side
            template='plotly_dark',  # For a sleek look
            coloraxis_showscale=False,  # No color scale needed
            xaxis_tickangle=-45  # Rotate x-axis labels for better visibility
        )

        # Show plot
        fig.show()

In [4]:
def save_results(results, dir):
    with open(dir, 'w') as f:
        json.dump(results, f, indent=4)

In [5]:
def read_result(dir):
    with open(dir, 'r') as f:
        data = json.load(f)
    return data

In [6]:
def scan_files_in_subdirs(directory):
    return [os.path.join(root, file) for root, _, files in os.walk(directory) for file in files]

In [28]:
def run(params: dict):
    args = ['/home/pedro/Pessoal/MSc/tip/main_prd']
    for p, v in params.items():
        args.append(f"--{p}={v}")
        
    result = subprocess.run(args, capture_output=True, text=True)
    return json.loads(result.stdout)

In [17]:
def run_all(runs: int = 1, instances_subset = []):
    def process_instance(instance, runs):
        instance_results = []
        for i in range(runs):
            params['filepath'] = instance
            params['rngSeed'] = random.randint(0, 1000000000)
            result = run(params)
            instance_results.append(result)
        return os.path.basename(instance), instance_results
    
    instances = scan_files_in_subdirs("/home/pedro/Pessoal/MSc/tip/instances")
    if(instances_subset):
        instances = [i for i in instances if os.path.basename(os.path.dirname(i)) in instances_subset]
        
    results = defaultdict(list)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_instance, instance, runs) for instance in instances]
        
        for future in tqdm.tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Running instances..."):
            instance_name, instance_results = future.result()
            results[instance_name].extend(instance_results)
    
    return results

In [31]:
results = run_all(runs=5, instances_subset=["y"])
save_results(results, "/home/pedro/Pessoal/MSc/tip/output/y_2.json")

Running instances...:  77%|███████▋  | 20/26 [04:55<01:28, 14.76s/it]


KeyboardInterrupt: 

In [27]:
result = read_result("/home/pedro/Pessoal/MSc/tip/output/s_2.json")
output_report = OutputReport(result)
output_report.plot_comparison()

In [10]:
output_report.plot_convergence()